# Data analysis

> Goal: Generate a list of top 300 subscription apps that had the best
rank growth over the past 365 days.

In [1]:
# read in data
from joblib import dump, load
import numpy as np
# for linear regression
from scipy.stats import linregress
import pandas as pd

In [2]:
app_info = load("app_info_df.pkl")

In [3]:
app_rank = load("app_rank_df.pkl")

In [4]:
app_info.head()

,app_name,developer,website,category,is_editor_choice,rating_oo5,num_ratings,has_iap,release_date,current_version,age_rating,file_size,editor_notes,description,os_compatibility,languages,price,itunes_link,date
itunes_app_id,,,,,,,,,,,,,,,,,,,
573984719,Advent Calendar: 25 Christmas Apps,MagicSolver,http://www.magicsolver.com/,Lifestyle,False,3.9,145,False,"Nov 24, 2015",1.1,Rated 4+,61.3 MB,NaN,\nDescription\n\n\nDiscover 25 of the finest f...,Requires iOS 7.0 or later. Compatible with iPh...,"English, Arabic, Dutch, French, German, Italia...",Free,https://itunes.apple.com/app/id573984719,2019-06-11-1512
424216726,Countdown‼,"Sevenlogics, Inc.",http://www.sevenlogics.com/mobile-apps/countdo...,Lifestyle,False,3.8,5500,True,"Jun 7, 2019",4.3.7,Rated 4+,152.7 MB,NaN,\nDescription\n\n\nCount down to your next eve...,Requires iOS 9.0 or later. Compatible with iPh...,"English, Danish, Dutch, Finnish, French, Germa...",Free,https://itunes.apple.com/app/id424216726,2019-06-11-1512
983764757,ShowOne - Pro Backing Tracks,One Zero One Audio,http://www.showoneapp.com,Music,False,4.4,41,True,"Jan 3, 2019",1.3.11,Rated 4+,63.7 MB,NaN,\nDescription\n\n\nShowOne is a pro app for mu...,Requires iOS 9.3 or later. Compatible with iPh...,English,Free,https://itunes.apple.com/app/id983764757,2019-06-11-1512
422370120,iPoll,"Instantly, Inc.",http://www.ipoll.com,Lifestyle,False,3.7,2100,False,"Apr 22, 2019",3.16.1,You must be at least 17 years old to download ...,80.6 MB,NaN,"\nDescription\n\n\nRewards. Anywhere, Anytime....",Requires iOS 10.0 or later. Compatible with iP...,English,Free,https://itunes.apple.com/app/id422370120,2019-06-11-1512
443052658,Surveys On The Go,"MFour Mobile Research, Inc",http://www.surveysonthego.net,Lifestyle,False,4.6,23100,False,"May 8, 2019",2.37,Rated 12+ for the following:,85.1 MB,NaN,"\nDescription\n\n\nFortune 500 companies, the ...",Requires iOS 10.0 or later. Compatible with iP...,English,Free,https://itunes.apple.com/app/id443052658,2019-06-11-1512


Look at the shape of the data. 

In [5]:
app_info.shape

(68834, 19)

In [6]:
app_rank.shape

(152249, 849)

Define a function to determine if an app is subscription-based by using keywords 'subscription', 'renewal','subscribe',"renew"

In [7]:
def subscription(description):
    subscribe_str_list = ['subscription', 'renewal','subscribe',"renew"]
    
    # if the description contains any of the words 
    # in subscribe_str_list, then return "Yes"
    for word in subscribe_str_list:
        if word in description:
            return "Yes"
        
    # if none of the words in subscribe_str_list is in description,
    # then return "No"   
    return "No"

If I was given more time, I can use word2vec to find similar words.  

Generate a column named `subscription` by using the `subscription` function. 

In [8]:
app_info['subscription'] = app_info['description'].apply(subscription)

In [9]:
app_info['subscription'].value_counts()

No     57721
Yes    11113
Name: subscription, dtype: int64

There is a total 11113 non-subscription app and 57721 subscription app. 

## Subset the `app_info` to get a df with all subscription-based apps. 

In [10]:
subscribe_app_index = app_info['subscription'] == "Yes"

In [11]:
subscribe_app_info = app_info[subscribe_app_index]

In [12]:
np.sort(subscribe_app_info.index.values)

array(['1000017994', '1000247107', '1000332606', ..., '999694732',
       '999769961', '999834966'], dtype=object)

In [13]:
app_rank.shape

(152249, 849)

In [14]:
subscribe_rank_df = app_rank.loc[subscribe_app_info.index]

In [15]:
subscribe_rank2_df = subscribe_rank_df.copy()

In [16]:
subscribe_rank_df.shape

(20587, 849)

Any rows that contains all NAN values? 

In [17]:
sum(subscribe_rank2_df.isna().sum(axis = 1) ==849)

0

Doing forward fill

In [18]:
subscribe_rank_df.head()

date                                                        2016-12-08  \
itunes_app_id rating_list                                                
1000017994    Health-and-fitness - topfreeapplications             NaN   
              Health-and-fitness - topgrossingapplications         NaN   
1000247107    Medical - topgrossingapplications                    NaN   
1000332606    Photo-and-Video - topfreeapplications                NaN   
              Photo-and-Video - topgrossingapplications            NaN   

date                                                        2016-12-09  \
itunes_app_id rating_list                                                
1000017994    Health-and-fitness - topfreeapplications             NaN   
              Health-and-fitness - topgrossingapplications         NaN   
1000247107    Medical - topgrossingapplications                    NaN   
1000332606    Photo-and-Video - topfreeapplications                NaN   
              Photo-and-Video - topgrossingapplications            NaN   

date                                                        2016-12-10  \
itunes_app_id rating_list                                                
1000017994    Health-and-fitness - topfreeapplications             NaN   
              Health-and-fitness - topgrossingapplications         NaN   
1000247107    Medical - topgrossingapplications                    NaN   
1000332606    Photo-and-Video - topfreeapplications                NaN   
              Photo-and-Video - topgrossingapplications            NaN   

date                                                        2016-12-11  \
itunes_app_id rating_list                                                
1000017994    Health-and-fitness - topfreeapplications           165.0   
              Health-and-fitness - topgrossingapplications         NaN   
1000247107    Medical - topgrossingapplications                    NaN   
1000332606    Photo-and-Video - topfreeapplications                NaN   
              Photo-and-Video - topgrossingapplications            NaN   

date                                                        2016-12-12  \
itunes_app_id rating_list                                                
1000017994    Health-and-fitness - topfreeapplications           191.0   
              Health-and-fitness - topgrossingapplications         NaN   
1000247107    Medical - topgrossingapplications                    NaN   
1000332606    Photo-and-Video - topfreeapplications                NaN   
              Photo-and-Video - topgrossingapplications            NaN   

date                                                        2016-12-13  \
itunes_app_id rating_list                                                
1000017994    Health-and-fitness - topfreeapplications             NaN   
              Health-and-fitness - topgrossingapplications         NaN   
1000247107    Medical - topgrossingapplications                    NaN   
1000332606    Photo-and-Video - topfreeapplications                NaN   
              Photo-and-Video - topgrossingapplications            NaN   

date                                                        2016-12-14  \
itunes_app_id rating_list                                                
1000017994    Health-and-fitness - topfreeapplications           197.0   
              Health-and-fitness - topgrossingapplications         NaN   
1000247107    Medical - topgrossingapplications                    NaN   
1000332606    Photo-and-Video - topfreeapplications                NaN   
              Photo-and-Video - topgrossingapplications            NaN   

date                                                        2016-12-15  \
itunes_app_id rating_list                                                
1000017994    Health-and-fitness - topfreeapplications           193.0   
              Health-and-fitness - topgrossingapplications         NaN   
1000247107    Medical - topgrossingapplications                    NaN 

In [19]:
clean_subscribe_rank_df = subscribe_rank2_df.ffill(axis = 1)

Get the last 365 days of the rank data. 

In [20]:
clean_subscribe_rank_df_365 = clean_subscribe_rank_df.iloc[:,-365:].copy()

The missing value here comes from the app does not exist in the app store yet. Fill the rest of missing values with 300

In [21]:
clean_subscribe_rank_df_365.fillna(300, inplace = True)

No more missing values.

In [22]:
sum(clean_subscribe_rank_df_365.isna().sum())

0

## get the growth rate through a simple linear regression

There are many approaches to compare growth rate among apps. I choose to do a simple linear regression on the rank data vs timestep. Then, the slope is extracted. A more negative value in slope means a quicker growth while a more positive slope means a decrease in rank. 

In [23]:
def get_growth_rate(rank_data):
    x = range(365)
    y = rank_data
    slope, intercept, r_value, p_value, std_err = linregress(x,y)
    if p_value < 0.05:
        return slope
    elif p_value >= 0.05:
        return 0    

In [24]:
growth_rate_list = [get_growth_rate(clean_subscribe_rank_df_365.iloc[i,]) for i in range(clean_subscribe_rank_df_365.shape[0])]

In [25]:
clean_subscribe_rank_df_365['growth_rate'] = growth_rate_list

Sort the dataframe by the growth rate and find out the top 300 apps that have the best growth in the past 365 days. 

In [26]:
growth_df = clean_subscribe_rank_df_365.sort_values(by = "growth_rate").iloc[:300,-2:]

In [27]:
app_info_growth_df = growth_df.join(subscribe_app_info , on = "itunes_app_id", how = "left")

In [28]:
app_info_growth_df.head()

2019-06-11  growth_rate  \
itunes_app_id rating_list                                                   
352509417     Newstand - topgrossingapplications         4.0    -1.216438   
              Newstand - topfreeapplications             4.0    -1.213315   
938922398     Newstand - topgrossingapplications         3.0    -1.213013   
1419779267    News - topgrossingapplications             9.0    -1.192442   
938922398     Newstand - topfreeapplications             8.0    -1.191679   

                                                                        app_name  \
itunes_app_id rating_list                                                          
352509417     Newstand - topgrossingapplications                 Washington Post   
              Newstand - topfreeapplications                     Washington Post   
938922398     Newstand - topgrossingapplications          Washington Post Select   
1419779267    News - topgrossingapplications      Fox Nation: Opinion Done Right   
938922398     Newstand - topfreeapplications              Washington Post Select   

                                                              developer  \
itunes_app_id rating_list                                                 
352509417     Newstand - topgrossingapplications    The Washington Post   
              Newstand - topfreeapplications        The Washington Post   
938922398     Newstand - topgrossingapplications    The Washington Post   
1419779267    News - topgrossingapplications      Fox News Network, LLC   
938922398     Newstand - topfreeapplications        The Washington Post   

                                                                                 website  \
itunes_app_id rating_list                                                                  
352509417     Newstand - topgrossingapplications          https://www.washingtonpost.com   
              Newstand - topfreeapplications              https://www.washingtonpost.com   
938922398     Newstand - topgrossingapplications  https://www.washingtonpost.com/mobile/   
1419779267    News - topgrossingapplications                  http://nation.foxnews.com/   
938922398     Newstand - topfreeapplications      https://www.washingtonpost.com/mobile/   

                                                 category is_editor_choice  \
itunes_app_id rating_list                                                    
352509417     Newstand - topgrossingapplications     News            False   
              Newstand - topfreeapplications         News            False   
938922398     Newstand - topgrossingapplications     News            False   
1419779267    News - topgrossingapplications         News            False   
938922398     Newstand - topfreeapplications         News            False   

                                                 rating_oo5 num_ratings  \
itunes_app_id rating_list                                                 
352509417     Newstand - topgrossingapplications        4.8      131300   
              Newstand - topfreeapplications            4.8      131300   
938922398     Newstand - topgrossingapplications        4.8       98000   
1419779267    News - topgrossingapplications            4.7        4900   
938922398     Newstand - topfreeapplications            4.8       98000   

                                                 has_iap  ...  \
itunes_app_id rating_list                                 ...   
352509417     Newstand - topgrossingapplications    True  ...   
              Newstand - topfreeapplications        True  ...   
938922398     Newstand - topgrossingapplications    True  ...   
1419779267    News - topgrossingapplications        True  ...   
938922398     Newstand - topfreeapplications        True  ...   

                                                                    age_rating  \
itunes_app_id rating_list                                                        
352509417     Newstand - topgrossingapplications  Rated 1

Write the data to disk. 

In [29]:
app_info_growth_df.to_csv("top300_subscription_app_info.csv")